In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

/home/ananth/anaconda2/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
conflict = pd.read_pickle('./me_events')

In [28]:

# conflict = pd.read_pickle('./saudi_events_kill_wound_sick')

def date_num(d):
        date = datetime.strptime(str(d), "%Y%m%d")
        return diff_days(date)
    
def date_unix(d):
    date = datetime.fromtimestamp(d)
    return diff_days(date)

def date_un(d):
    d = str(d)
    d = d[:-2] + '20' + d[-2:]
    date = datetime.strptime(d, "%d-%m-%Y")
    return diff_days(date)

def date_upps(d):
    date = datetime.strptime(d, "%m/%d/%Y %H:%M:%S")
    return diff_days(date)

def diff_days(date):
    base_date = datetime.strptime('20130401', '%Y%m%d')
    diff = date - base_date
    return diff.days

def get_ts_event(eventtype):
    kills = conflict[conflict['COUNTTYPE']==eventtype]
    def sanitize(r):
        x = int(r)
        if x > 10000:
            return 0
        return x
        
    kills['NUMBER'] = kills['NUMBER'].apply(lambda row: sanitize(row))
    kills['DATE'] = kills['DATE'].apply(lambda d: date_num(d))
    kills_g = kills.groupby(['DATE', 'GEO_COUNTRYCODE'])['NUMBER'].sum()
#     kills_g = kills.groupby(['DATE'])['NUMBER'].sum()
    return kills_g.reset_index()

In [3]:
killed = get_ts_event('KILL')
wounded = get_ts_event('WOUND')
sick_news = get_ts_event('SICKENED')
affected = get_ts_event('AFFECT')


# killed.corr(wounded)
# sick_news.corr(killed)

# join_kill_sick = killed.join(sick_news.set_index('DATE'), on='DATE', lsuffix='_killed', rsuffix='_sick', how='outer')
# join_kill_sick.fillna(0, inplace=True)
# # join_kill_sick
# grangercausalitytests(join_kill_sick[['NUMBER_sick','NUMBER_killed']].as_matrix(), maxlag=10)
# join_kill_sick[['NUMBER_sick','NUMBER_killed']].as_matrix()

/home/ananth/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ananth/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
dates = {'DATE': range(1736)}
dates = pd.DataFrame(data=dates)

all_series = dates
events = {'kill':killed, 'wound':wounded, 'sick':sick_news, 'affect':affected}
for name, ts in events.items():
    for c in np.unique(ts['GEO_COUNTRYCODE']):
        ts_c = ts[ts['GEO_COUNTRYCODE']==c][['DATE', 'NUMBER']]
        ts_joined = dates.join(ts_c.set_index('DATE'), on='DATE', how='outer')
        ts_joined = pd.DataFrame({'{0}_{1}'.format(name, c) : ts_joined.fillna(0)['NUMBER'].as_matrix()})
        all_series = pd.concat([all_series, ts_joined], axis=1)
all_series

,DATE,kill_AE,kill_BA,kill_EG,kill_IR,kill_JO,kill_KU,kill_LE,kill_MU,kill_QA,...,affect_EG,affect_IR,affect_JO,affect_KU,affect_LE,affect_MU,affect_QA,affect_SA,affect_TS,affect_YM
0,0,250.0,72.0,6048.0,122.0,5740.0,115.0,22706.0,0.0,0.0,...,0.0,0.0,89.0,0.0,71.0,200.0,0.0,2018.0,100.0,0.0
1,1,10.0,24.0,811.0,4582.0,0.0,69.0,2000.0,0.0,0.0,...,1097.0,0.0,3423.0,0.0,0.0,0.0,0.0,2000.0,0.0,10502.0
2,2,5.0,44.0,342.0,275.0,26.0,0.0,6024.0,0.0,603.0,...,583.0,4.0,0.0,0.0,344.0,0.0,0.0,7006.0,0.0,0.0
3,3,0.0,104.0,2.0,413.0,0.0,10.0,20.0,0.0,638.0,...,0.0,0.0,17569.0,2.0,2117.0,270.0,0.0,2.0,7.0,0.0
4,4,17.0,0.0,55.0,2821.0,2029.0,45.0,41.0,0.0,0.0,...,570.0,21.0,8201.0,0.0,4508.0,0.0,0.0,9.0,14.0,0.0
5,5,0.0,0.0,138.0,88.0,3.0,5.0,15.0,730.0,0.0,...,500.0,15.0,2010.0,10.0,78.0,0.0,0.0,0.0,0.0,1990.0
6,6,6.0,0.0,1610.0,25.0,80.0,0.0,10.0,0.0,2.0,...,24.0,2031.0,404.0,0.0,10024.0,0.0,0.0,0.0,0.0,0.0
7,7,8.0,94.0,247.0,2033.0,70.0,0.0,27015.0,0.0,44.0,...,0.0,32.0,70.0,0.0,2100.0,0.0,4.0,0.0,14.0,9.0
8,8,446.0,103.0,1313.0,2372.0,1992.0,0.0,10145.0,0.0,33.0,...,24.0,20249.0,84.0,0.0,50.0,0.0,0.0,2000.0,7.0,3.0
9,9,50.0,279.0,5110.0,1834.0,168.0,0.0,5.0,0.0,0.0,...,1061.0,25050.0,5554.0,0.0,5.0,0.0,0.0,0.0,7.0,0.0


In [5]:
diff = pd.read_csv('diff.csv')
diff['DATE'] = diff['DATE'].apply(lambda x: date_un(x))
diff['CASES'].corr(diff['DEATHS'])

0.53375891032715483

In [41]:
all_upps =pd.DataFrame()
for y in [2013,2014,2015,2016]:
    upps = pd.read_csv('Armed Conflict/gedevents-{0}.csv'.format(y))
    upps_y = upps[(upps['country']==616) | (upps['country']==630) | (upps['country']==651) | (upps['country']==660) 
         | (upps['country']==663) | (upps['country']==670) | (upps['country']==678) | (upps['country']==690)][['country', 'date_start', 'best_est']]
    all_upps = pd.concat([all_upps, upps_y], axis=0)

all_upps['DATE'] = all_upps['date_start'].apply(lambda x: date_upps(x))
all_upps = all_upps.drop(['date_start'], axis=1)

dates = {'DATE': range(1736)}
dates = pd.DataFrame(data=dates)
all_series_upps = dates
for c in np.unique(all_upps['country']):
    all_upps_c = all_upps[all_upps['country']==c][['DATE', 'best_est']]
    ts_joined = dates.join(all_upps_c.set_index('DATE'), on='DATE', how='right')
    ts_joined = pd.DataFrame({'death_{0}'.format(c) : ts_joined['best_est'].as_matrix()})
    all_series_upps = pd.concat([all_series_upps, ts_joined], axis=1)
all_series_upps.fillna(0, inplace=True)
all_series_upps

,DATE,death_616,death_630,death_651,death_660,death_663,death_670,death_678,death_690
0,0,21.0,9.0,5.0,2.0,1.0,3.0,1.0,26.0
1,1,38.0,6.0,2.0,4.0,8.0,1.0,1.0,0.0
2,2,0.0,3.0,3.0,4.0,7.0,1.0,4.0,0.0
3,3,0.0,1.0,1.0,3.0,14.0,1.0,1.0,0.0
4,4,0.0,2.0,1.0,7.0,5.0,3.0,5.0,0.0
5,5,0.0,4.0,2.0,1.0,0.0,5.0,7.0,0.0
6,6,0.0,5.0,1.0,4.0,0.0,2.0,1.0,0.0
7,7,0.0,2.0,1.0,2.0,0.0,16.0,3.0,0.0
8,8,0.0,8.0,2.0,6.0,0.0,1.0,1.0,0.0
9,9,0.0,3.0,2.0,1.0,0.0,1.0,1.0,0.0


In [58]:
from sklearn.preprocessing import StandardScaler
def get_all_vars(filename, series):
    model = pd.read_csv(filename)
    model['DATE'] = model['Timestamp'].apply(lambda x: date_un(x))
    all_vars_model = model.join(series.set_index('DATE'), on='DATE', how='inner')
    all_vars_model = all_vars_model.drop(['Timestamp', 'DATE', 'SIR Values', 'True Values', 'RSS'], axis=1)

    mat_scaled = StandardScaler()
    mat_scaled.fit(all_vars_model)
    mat_sc = mat_scaled.transform(all_vars_model)
    all_vars_model = pd.DataFrame(mat_sc, columns=all_vars_model.columns)
    cols = all_vars_model.columns.values
    cols[1], cols[0] = cols[0], cols[1]
    all_vars_model = all_vars_model[cols]
    all_vars = all_vars_model.as_matrix()
    return all_vars

# #GDELT
# all_vars = get_all_vars('model_1.csv', all_series)
# all_vars_test = get_all_vars('model_2.csv', all_series)

#Uppsala
all_vars = get_all_vars('model_1.csv', all_series_upps)
all_vars_test = get_all_vars('model_2.csv', all_series_upps)

# #Uppsala+GDELT
# joined_data = all_series_upps.join(all_series.set_index('DATE'), on='DATE', how='outer')
# all_vars = get_all_vars('model_1.csv', joined_data)
# all_vars_test = get_all_vars('model_2.csv', joined_data)

In [59]:
import stattools
# import importlib
# importlib.reload(stattools)

error, coeff, best_vars = stattools.grangercausalitytests(all_vars, all_vars_test, mxlg=6, alpha=1e-4)
error


Granger Causality
number of lags (no zero) 6


126.13630424237279

In [148]:
join_kill_sick['NUMBER_sick'].corr(join_kill_sick['NUMBER_killed'])

0.18695775760972388

In [152]:
from matplotlib import pyplot as plt
plt.clf()

# t = join_kill_sick['DATE']
# data1 = join_kill_sick['NUMBER_sick']
# data2 = join_kill_sick['NUMBER_killed']

t = diff['DATE']
data1 = diff['CASES']
data2= diff['DEATHS']

fig, ax1 = plt.subplots()

color = 'red'
ax1.set_xlabel('time (days)')
ax1.set_ylabel('cases', color=color)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
# ax2 = ax1

color = 'blue'
ax2.set_ylabel('deaths', color=color)  # we already handled the x-label with ax1
ax2.plot(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
# plt.show()
plt.savefig('mers-cases-deaths.png')
# plt.savefig('gdelt-sick-killed.png')

In [115]:
# sir = pd.read_csv('SIR results.csv')
# sir['TimeStamp'] = sir['TimeStamp'].apply(lambda x: date_unix(x))
# print(sir)

un_sick = pd.read_csv('death_count4.csv')
un_sick = un_sick[['DATE', 'COUNTRY', 'CASES', 'DEATHS']]
# # un_sick = un_sick[un_sick['COUNTRY']=='Saudi Arabia']
un_sick['DATE'] = un_sick['DATE'].apply(lambda x: date_un(x))
print(un_sick)

     DATE                        COUNTRY  CASES  DEATHS
0    -597                   Saudi Arabia     35      20
1    -596                   Saudi Arabia     39      20
2    -593                   Saudi Arabia     41      20
3    -589                        Tunisia     43      20
4    -588  Jordan/Qatar/Saudi Arabia/UAE     44      22
5    -588      France/Germany/Tunisia/UK     45      22
6    -582                   Saudi Arabia     49      22
7    -580  Jordan/Qatar/Saudi Arabia/UAE     50      30
8    -579  Jordan/Qatar/Saudi Arabia/UAE     51      30
9    -578                         Jordan     53      30
10   -575         Qatar/Saudi Arabia/UAE     54      30
11   -573  Jordan/Qatar/Saudi Arabia/UAE     55      31
12   -566                   Saudi Arabia     58      32
13   -565                   Saudi Arabia     61      33
14   -563                   Saudi Arabia     64      33
15   -558                   Saudi Arabia     68      33
16   -557                   Saudi Arabia     70 